In [20]:
from pathlib import Path

import pandas as pd
from tqdm import tqdm
import numpy as np
import SimpleITK as sitk

In [3]:
pt_files = [f for f in Path("../data/interim/nii_raw").rglob("*PT.nii.gz")]

In [16]:
gtvl_files = [f for f in Path("../data/interim/nii_raw").rglob("*GTV_L*")]
gtvt_files = [f for f in Path("../data/interim/nii_raw").rglob("*GTV_T*")]

In [11]:
resampler = sitk.ResampleImageFilter()
resampler.SetInterpolator(sitk.sitkNearestNeighbor)


In [29]:
df = pd.DataFrame()
for f in tqdm(pt_files):
    patient_id = f.name.split("__")[0]
    pt = sitk.ReadImage(str(f))
    gtvl_file = [ff for ff in gtvl_files if patient_id + "__" in ff.name][0]
    gtvt_file = [ff for ff in gtvt_files if patient_id + "__" in ff.name][0]
    gtvl = sitk.ReadImage(str(gtvl_file))
    gtvt = sitk.ReadImage(str(gtvt_file))
    resampler.SetReferenceImage(gtvt)
    pt = resampler.Execute(pt)
    gtvl = resampler.Execute(gtvl)
    gtvl = sitk.GetArrayFromImage(gtvl)
    gtvt = sitk.GetArrayFromImage(gtvt)
    pt = sitk.GetArrayFromImage(pt)
    try:
        suvmax_gtvl = np.max(pt[gtvl != 0])
        suvmax_gtvt = np.max(pt[gtvt != 0])
    except Exception as e:
        print(f"Error with patient {patient_id}")
        continue

    df = df.append(
        {
            "patient_id": patient_id,
            "suv_max_gtvl": suvmax_gtvl,
            "suv_max_gtvt": suvmax_gtvt,
        },
        ignore_index=True)


100%|██████████| 109/109 [06:24<00:00,  3.53s/it]


In [28]:
df.sort_values(by="patient_id").to_csv("manual_suvmax.csv")